#prev question from lab5 

In [31]:
import torch 
from torch import nn
from torch.utils.data import DataLoader,Dataset
import torchvision 
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.FashionMNIST(root='/data',download=True,train=True,transform=ToTensor(),target_transform=None)
test_data = datasets.FashionMNIST(root='/data',download=True,train=False,transform=ToTensor()) 
batch_size = 32 
train_dataloader = DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(dataset=test_data,batch_size=batch_size,shuffle=True)
class_names = train_data.classes
train_features_batch,train_labels_batch = next(iter(train_dataloader))

In [32]:
torch.manual_seed(42)
rand_idx = torch.randint(0,len(train_features_batch),size=[1]).item()
img,label = train_features_batch[rand_idx],train_labels_batch[rand_idx]

In [33]:
class CNNclf(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=3),
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2)
        )

        self.clf = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64,20,bias=True),
            nn.ReLU(),
            nn.Linear(20,10,bias=True)
        )

    def forward(self,x):
        x = self.net(x)
        x = self.clf(x)
        return x
    
torch.manual_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CNNclf().to(device)
print(model)

CNNclf(
  (net): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=3, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (clf): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=64, out_features=20, bias=True)
    (2): ReLU()
    (3): Linear(in_features=20, out_features=10, bias=True)
  )
)


In [34]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),lr=0.001)

In [35]:
def accuracy_fn(y_true,y_pred):
    correct = torch.eq(y_pred,y_true).sum().item()
    acc = (correct/len(y_pred)) * 100
    return acc

In [36]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    model.to(device)
    train_loss,train_acc = 0,0
    for batch,(X,y) in enumerate(data_loader):
        X,y = X.to(device),y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        train_acc += accuracy_fn(y,y_pred.argmax(dim=1))
        train_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} Train accuracy: {train_acc:.2f}%")


def test_step(model,data_loader,loss_fn,optimizer,accuracy_fn,device):
    test_loss,test_acc = 0,0
    model.to(device)
    model.eval()
    with torch.inference_mode():
        for X,y in data_loader:
            X,y = X.to(device),y.to(device)
            y_pred = model(X)
            test_loss += loss_fn(y_pred,y)
            test_acc += accuracy_fn(y,y_pred.argmax(dim=1))
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
    print(f"Test loss: {test_loss:.5f} Test accuracy: {test_acc:.2f}%")

In [37]:
from timeit import default_timer as timer 
def print_train_time(start: float, end: float, device: torch.device = None):
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time

In [38]:
torch.manual_seed(42)
from tqdm import tqdm
# Measure time
from timeit import default_timer as timer
train_time_start_on_gpu = timer()

epochs = 3
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train_step(data_loader=train_dataloader, 
        model=model, 
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn
    )
    test_step(data_loader=test_dataloader,
        model=model,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn,
        optimizer=optimizer,
        device=device
    )

train_time_end_on_gpu = timer()
total_train_time_model = print_train_time(start=train_time_start_on_gpu,
                                            end=train_time_end_on_gpu,
                                            device=device)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 0.57454 Train accuracy: 78.59%


 33%|███▎      | 1/3 [00:08<00:17,  8.66s/it]

Test loss: 0.40318 Test accuracy: 85.38%
Epoch: 1
---------
Train loss: 0.35287 Train accuracy: 87.08%


 67%|██████▋   | 2/3 [00:17<00:08,  8.71s/it]

Test loss: 0.34717 Test accuracy: 87.56%
Epoch: 2
---------
Train loss: 0.29356 Train accuracy: 89.29%


100%|██████████| 3/3 [00:26<00:00,  8.75s/it]

Test loss: 0.30902 Test accuracy: 88.86%
Train time on cuda: 26.260 seconds


Lab-6
Question 1

Perform classification on FashionMNIST, fashion apparels dataset, using a pre-trained model which is trained on MNIST handwritten digit classification dataset.

In [39]:
torch.save(model,"./ModelFiles/model.pt")

In [40]:
model = CNNclf()
model = torch.load("./ModelFiles/model.pt")
model.to(device)

CNNclf(
  (net): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=3, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (clf): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=64, out_features=20, bias=True)
    (2): ReLU()
    (3): Linear(in_features=20, out_features=10, bias=True)
  )
)

In [41]:
print("model's state_dict")
for param_tensor in model.state_dict().keys():
    print(param_tensor, "\t",model.state_dict()[param_tensor].size())
print()

model's state_dict
net.0.weight 	 torch.Size([64, 1, 3, 3])
net.0.bias 	 torch.Size([64])
net.3.weight 	 torch.Size([128, 64, 3, 3])
net.3.bias 	 torch.Size([128])
net.6.weight 	 torch.Size([64, 128, 3, 3])
net.6.bias 	 torch.Size([64])
clf.1.weight 	 torch.Size([20, 64])
clf.1.bias 	 torch.Size([20])
clf.3.weight 	 torch.Size([10, 20])
clf.3.bias 	 torch.Size([10])



In [44]:
torch.manual_seed(42)
from timeit import default_timer as timer 
from tqdm import tqdm

epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch}\n-------")
    test_step(model, test_dataloader, loss_fn, optimizer, accuracy_fn, device)

test_time_end_on_gpu = timer()
total_test_time_model = print_train_time(start=train_time_start_on_gpu,
                                            end=train_time_end_on_gpu,
                                            device=device)


Epoch 0
-------
Test loss: 0.30858 Test accuracy: 88.88%
Epoch 1
-------
Test loss: 0.30909 Test accuracy: 88.87%
Epoch 2
-------
Test loss: 0.30833 Test accuracy: 88.89%
Train time on cuda: 26.260 seconds


2. Learn the AlexNet architecture and apply transfer learning to perform the classification task. Using the pre-trained AlexNet, classify images from the cats_and_dogs_filtered dataset downloaded from the below link. Finetune the classifier given in AlexNet as a two-class classifier. Perform pre-processing of images as per the requirement.

In [52]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import os

# Define data directory paths
train_data_dir = 'Lab-6/cats_and_dogs_filtered/train'
test_data_dir = 'Lab-6/cats_and_dogs_filtered/validation'

# Define transformation for data preprocessing
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
}

# Create datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(train_data_dir if x == 'train' else test_data_dir),
                                          data_transforms[x])
                  for x in ['train', 'test']}

# Create data loaders
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32,
                             shuffle=True, num_workers=4)
               for x in ['train', 'test']}

# Load pre-trained AlexNet
alexnet = models.alexnet(pretrained=True)

# Freeze convolutional layers
for param in alexnet.parameters():
    param.requires_grad = False

# Modify the last fully connected layer for binary classification
num_features = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Linear(num_features, 2)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(alexnet.parameters(), lr=0.001)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

# Training loop
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

# Train the model
train_model(alexnet, criterion, optimizer, num_epochs=10)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Lab-6/cats_and_dogs_filtered/train'